# Prep 1: steps to install the packages

In [ ]:
!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

In [183]:
import py_entitymatching as em
import pandas as pd
import os

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [465]:
table_a = 'TableA'
table_b = 'TableB'
candidate_set = 'CandidateSet'
prediction_set = 'PredictionSet'

# Prep 3: reading the files into pandas dataframe

In [466]:
dfa = em.read_csv_metadata(table_a, key='_id')
dfa['Name'] = dfa['Name'].str.lower()

dfb = em.read_csv_metadata(table_b, key='_id')
dfb['Name'] = dfb['Name'].str.lower()

dfc = em.read_csv_metadata(candidate_set, key='_id',
                           fk_ltable='A_id', fk_rtable='B_id',
                           ltable=dfa, rtable=dfb)
dfp = em.read_csv_metadata(prediction_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [447]:
block_f = em.get_features_for_blocking(dfa, dfb, validate_inferred_attr_types=False)
block_f = block_f.iloc[[4],:]

# Code for reducing density

In [448]:
rb = em.RuleBasedBlocker()
rb.add_rule(['Name_Name_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5'], block_f)

'_rule_0'

In [449]:
C2 = rb.block_candset(dfc)
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


491

In [450]:
C2.to_csv('C3.csv', index=False)
C3 = 'C3.csv'

In [455]:
densityFlag = False
i = 0
C = dfa
L = []
while densityFlag == False:
    B = em.sample_table(C2, 50)
    B.to_csv('B.csv', index=False)
    densityFlag=True

,_id,A_id,B_id
95,95,1058,728
1415,1415,1661,2187
2708,2708,2249,728
4319,4319,2999,3710
5665,5665,3740,1798
7055,7055,446,2692
8130,8130,557,2946
9095,9095,1059,257
13950,13950,3444,3070
14627,14627,3768,4569


# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [ ]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [456]:
import py_entitymatching as em
import pandas as pd

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr, output_size=200)
    
    return debug_file

In [457]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [476]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [477]:
# read the labeled pairs file, i.e. the file with the labels
red_cand_set = 'reduced_cands.csv'
# dfc = em.read_csv_metadata(candidate_set)
dfc = em.read_csv_metadata(red_cand_set,
                           fk_ltable='id1', fk_rtable='id2',
                           ltable=dfa, rtable=dfb)


labeled_pairs = em.read_csv_metadata('labeled_pairs.csv')
print(estimate_PR(labeled_pairs, dfc, dfp))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


((0.983121531366644, 0.9932796485743588), (0.9828007156935782, 0.9925687424345006))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb